# Instrumental Variables

This notebook displays how Instrumental Variables can be introduced to correctly estimate the true $\beta$ in $Y = \beta X + \epsilon$, when an explanatory variable ($X$) is correlated with the error term ($\epsilon$). In this case, we can introduce a new variable $Z$, which induces changes in the explanatory variable ($X$) but has no independent effect on the dependent variable (no correlation with $\epsilon$).

In the visualization below, $corr(X, \epsilon) = 0.7$, $corr(X, Z)$ is controllable with the slidebar, and $corr(Z, \epsilon)=0$ to correctly display the effect of introducting $Z$.

To create the visualization, click on the code block below, and run that block by clicking on the 'run cell' button marked with red on the following image.

![title](Menu.PNG)

Running the code cell for the first time may take a while. Once the run is complete, you'll see some slidebars/dropdown menus along with the 'Run Interact' button. Adjust the parameters, and then press the button to see the visualization.

In [2]:
import ipywidgets as widgets
from visfunctions import IV
from ipywidgets import interact_manual

interact_manual(IV, covXZ=(-0.9, 0.9))

<function visfunctions.IV>